<a href="https://colab.research.google.com/github/Andrian0s/ML4NLP1-2024-Tutorial-Notebooks/blob/main/exercises/ex5/ex5_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 5: LLM Prompting and Prompt Engineering Part 2

In part 2, we experiment with prompting instruction-tuned Large Language Models (LLMs), and evaluate their performance on a linguistic annotation task involving structured outputs.

The goal of this assignment is to gain some experience working with instruction-tuned LLMs. To this end, you will learn how to

- query an instruction-tuned LLM with default chat templates (`Llama-3.2-3B-Instruct`)
- parse LLM outputs for structured responses using JSON and `Pydantic`
- implement error handling for edge cases where the model fails to output the expected data format.

The task we use for this purpose is a simple Tokenization and Part-of-Speech tagging task using data taken from Universal Dependencies.

To facilitate working with LLMs, we will again use the Unsloth library. Note that Unsloth provides both freeware and closed-source proprietary software. For our purposes, the freeware is sufficient! For more information on Unsloth, see the docs here.

This notebook is adapted from [this example](https://colab.research.google.com/drive/1T5-zKWM_5OD21QHwXHiV9ixTRR7k3iB9?usp=sharing) by Unsloth.


### NOTE: Expected execution times
We have provided expected execution times throughout the notebook as a guide. These are intended to be approximate, but should give you some idea for what to expect. If your runtimes far exceed these expected execution times, you may want to consider modifying your approach. These are denoted with ⌛ .

### NOTE: GPU Usage
It is expected that you load the model onto a GPU for inference. For other parts of the code, such as data preparation, a GPU is not necessary. To avoid waiting for resources unnecessarily, we recommend doing as much as you can on a CPU instance and change the runtime type as necessary. We've highlight the cells that need a GPU with ⚡

## 1) Installing dependencies

In [ ]:
%%capture
!pip install levenshtein
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# check unsloth version
expected_version = '2024.10.2'
unsloth_version = !pip list | grep -P 'unsloth\s+' | grep -Po '\S+$'
if unsloth_version[0] != expected_version:
    print(f"Warning! Found Unsloth version {unsloth_version[0]} but expected {expected_version}.")

# check python version
import sys
print(sys.version)

# check gpu info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# check RAM info
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## 2) Model Loading

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# Note, here we specify the instruction-tuned version of Llama-3.2-3B
model_name = "unsloth/Llama-3.2-3B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )


FastLanguageModel.for_inference(model) # Enable native 2x faster inference

## 3) Data Loading and Preparation

In [ ]:
# load data
import random
import pandas as pd

seed = 42

random.seed(seed)

dataset_url = "https://raw.githubusercontent.com/tannonk/prompting_exercise/refs/heads/main/data/en_ewt-ud-dev-pos.json"
df = pd.read_json(dataset_url, lines=True)

# For each input sentence, we'll build the target as a list of dictionaries containing keys for the token and pos tag. This is what we want our LLM annotator to predict.
df['target'] = df.apply(lambda x: [{'token': token, 'pos': pos} for token, pos in zip(x['tokens'].split(), x['upos'].split())], axis=1)

# We'll sample 100 items for testing purposes
test_data = df.sample(n=100, random_state=seed)
train_data = df.drop(test_data.index)

print(f"Train data: {len(train_data)}")
print(f"Test data: {len(test_data)}")

test_data.head()


### TODO: Inspect and describe the data

📝❓ What are the fields and their corresponding values in the dataframe?

📝❓ What is the difference between `upos` and `xpos`?

📝❓ What is the distribution of `upos` labels in the `test_data`?

### TODO: Define the basic `PromptTemplate`

Note, you can reuse the solution from part 1 of this exercise here.

In [ ]:
# TODO

## 3.2 ChatTemplates

Instruction-tuned models are typically finetuned using a predefined `ChatTemplate`.
This means that when using them for inference, it is important that we use the correct `ChatTemplate` in order to avoid "confusing" the model.

You can find more information about model `ChatTemplates` for Huggingface models [here](https://huggingface.co/docs/transformers/en/chat_templating).


In [ ]:
from unsloth.chat_templates import get_chat_template

# load the chat_template from unsloth (note, the logic is similar when using native Huggingface, but here we're using Unsloth!)
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1", # for Llama-3.1 and Llama-3.2 models
)

# Inspect the template (note, it looks more complicated than it is!)
print(tokenizer.chat_template)


### TODO: Prepare your inputs using the `ChatTemplate` for the model.

Note, you should be able to drop your custom `PromptTemplate` string into the model's default `ChatTemplate`.


In [ ]:
# TODO

## 4) Inference Pipeline

### TODO: Define a function to run inference efficiently with an LLM

Note, you can use the same inference function from part 1 of this exercise here!

In [ ]:
# Set up our inference pipeline for generation

# We'll set some default generation args that we'll pass to our inference function
# Following best practices, we'll use Pydantic class which helps with validation.
from pydantic import BaseModel

class Generation_Args(BaseModel):
    max_new_tokens: int
    temperature: float
    top_k: int
    top_p: float
    repetition_penalty: float
    do_sample: bool
    min_p: float
    num_return_sequences: int

# Here are some default generation args
generation_args = Generation_Args(
    max_new_tokens = 1024, # note, for this task, we're setting the max_new_tokens to be more appropriate
    temperature = 1.0,
    top_k = 0,
    top_p = 1.0,
    repetition_penalty = 1.0,
    do_sample = True,
    use_cache = True,
    min_p = 0.1,
    num_return_sequences = 1
)


def run_batched_inference(prompts, model, tokenizer, batch_size=10, generation_args=generation_args):
    """
    Runs batched inference on a list of prompts using a given model and tokenizer.

    Set the batch_size to control the number of prompts processed in each batch.
    Depending on the length of your prompts and model size the batch size may need to be adjusted.

    Args:
        prompts (list[str]): List of prompts that are passed to the model
        model (): The model used for generation
        tokenizer (): The tokenizer used for encoding and decoding the prompts
        batch_size (int): Number of prompts to run batched inference on.

    Returns:
        List[str] containing generated outputs.
    """

    outputs = []

    # TODO: implement the logic for efficient inference with LLM

    return outputs

## 4.1) Run Inference

### TODO: run inference!

⌛ 10-20 mins

⚡ GPU

In [ ]:
# TODO: inference

## 5) Structured output validation

LLMs output text. But in practice, we often want structured data that we can process further with other automatic processes.

For this purpose, JSON is a good target data structure.


### TODO: Define a processing pipeline that extracts and validates the JSON response from the LLM.

Hint: For this you should use a combination of [`Regex`](https://www.w3schools.com/python/python_regex.asp) and [`Pydantic`](https://docs.pydantic.dev/latest/).

The output should be a valid json object with the following structure:

```json
[
    {"token": "there", "pos": "DET"}, # each dict contains a token and its corresponding POS-Tag.
    {"token": "is", "pos": "VERB"},
    {"token": "no", "pos": "ADJ"},
    {"token": "companion", "pos": "NOUN"},
    {"token": "quite", "pos": "ADV"},
    {"token": "so", "pos": "ADV"},
    {"token": "devoted", "pos": "ADV"},
    {"token": "so", "pos": "ADV"},
    ...
]
```

In [1]:
# TODO

## 6) *Evaluation*

In [ ]:
# Below is some boilerplate evaluation code. You should not need to make any changes here.

import Levenshtein
import numpy as np
from typing import List, Dict

def evaluate_instance(target: List[Dict], prediction: List[Dict]):
    """
    Evaluates the accuracy of tokenization and part-of-speech (POS) tagging between a target and a predicted sequence.

    Args:
        target (List[Dict]): A list of dictionaries representing the target tokens and POS tags.
        prediction (List[Dict]): A list of dictionaries representing the predicted tokens and POS tags.

    Returns:
        dict: A dictionary containing the token-level accuracy ('Token Acc') and POS accuracy ('POS Acc').
    """

    # If there is no prediction, return zero accuracies
    if prediction is None:
        return {'Token Acc': 0, 'POS Acc': 0}

    # Extract tokens and POS tags from the target and prediction lists
    target_tokens = [item['token'] for item in target]
    target_pos = [item['pos'] for item in target]
    pred_tokens = [item['token'] for item in prediction]
    pred_pos = [item['pos'] for item in prediction]

    # Get alignment operations between the target and predicted tokens using Levenshtein.opcodes()
    opcodes = Levenshtein.opcodes(target_tokens, pred_tokens)

    # Initialize aligned lists to store tokens and POS tags after alignment
    aligned_target_tokens = []
    aligned_target_pos = []
    aligned_pred_tokens = []
    aligned_pred_pos = []

    # Iterate over each operation in the alignment
    for tag, i1, i2, j1, j2 in opcodes:
        # "equal" means the tokens in this range are identical in both sequences
        if tag == 'equal':
            aligned_target_tokens.extend(target_tokens[i1:i2])
            aligned_target_pos.extend(target_pos[i1:i2])
            aligned_pred_tokens.extend(pred_tokens[j1:j2])
            aligned_pred_pos.extend(pred_pos[j1:j2])
        # "replace" means tokens in this range are different between the target and prediction
        elif tag == 'replace':
            aligned_target_tokens.extend(target_tokens[i1:i2])
            aligned_target_pos.extend(target_pos[i1:i2])
            aligned_pred_tokens.extend(pred_tokens[j1:j2])
            aligned_pred_pos.extend(pred_pos[j1:j2])
        # "insert" means tokens were added in the prediction that are not in the target
        elif tag == 'insert':
            aligned_target_tokens.extend(['<MISSING>'] * (j2 - j1))  # Add placeholders for missing target tokens
            aligned_target_pos.extend(['<MISSING>'] * (j2 - j1))      # Add placeholders for missing target POS tags
            aligned_pred_tokens.extend(pred_tokens[j1:j2])
            aligned_pred_pos.extend(pred_pos[j1:j2])
        # "delete" means tokens are present in the target but missing in the prediction
        elif tag == 'delete':
            aligned_target_tokens.extend(target_tokens[i1:i2])
            aligned_target_pos.extend(target_pos[i1:i2])
            aligned_pred_tokens.extend(['<MISSING>'] * (i2 - i1))    # Add placeholders for missing predicted tokens
            aligned_pred_pos.extend(['<MISSING>'] * (i2 - i1))       # Add placeholders for missing predicted POS tags

    # Calculate token-level accuracy
    # We only consider positions where both target and prediction have valid tokens (i.e., not '<MISSING>')
    correct_tokens = [
        1 if tgt == pred else 0
        for tgt, pred in zip(aligned_target_tokens, aligned_pred_tokens)
        if tgt != '<MISSING>' and pred != '<MISSING>'
    ]
    token_accuracy = np.mean(correct_tokens) if correct_tokens else 0

    # Calculate POS accuracy
    # Only consider positions where tokens match and are not '<MISSING>'
    correct_pos = [
        1 if tgt_pos == pred_pos else 0
        for tgt_tok, pred_tok, tgt_pos, pred_pos in zip(aligned_target_tokens, aligned_pred_tokens, aligned_target_pos, aligned_pred_pos)
        if tgt_tok == pred_tok and tgt_tok != '<MISSING>'
    ]
    pos_accuracy = np.mean(correct_pos) if correct_pos else 0

    return {'Token Acc': token_accuracy, 'POS Acc': pos_accuracy}

def get_results(test_data: pd.DataFrame, processed_outputs: List[List[Dict]]):
    """
    Returns a summary dataframe by taking the average of the all results for tokenization and pos-tagging.
    """
    results = []
    for i in range(len(processed_outputs)):
        results.append(evaluate_instance(test_data.iloc[i]['target'], processed_outputs[i]))

    results = pd.DataFrame(results).mean()
    return results

In [ ]:
# To get the results, you should be able to pass your test_data DataFrame and the processed_outputs from above...
get_results(test_data, processed_outputs)

## 7) Manipulating the system prompt

The system prompt is part of the `ChatTemplate` that can help to steer the model.


### TODO: Customise the system prompt for the intended task and re-run inference

Note, this is an experiment. You should try a few different system prompts and report the resulting performance in your report.


📝❓ What was the best system prompt you considered?

📝❓ Were you able to improve the performance by manipulating the system prompt? Please discuss.

⌛ 10-20 mins (per experiment run)

⚡ GPU

In [ ]:
# TODO: inference

---

## 8) Lab report

📝❓ Write your lab report here addressing all questions in the notebook